In [ ]:
#Custom URL
#https://llama3-1.llamameta.net/*?Policy=eyJTdGF0ZW1lbnQiOlt7InVuaXF1ZV9oYXNoIjoiN3VzdDc2amxub3F1NXA4aXRyY2U5eW40IiwiUmVzb3VyY2UiOiJodHRwczpcL1wvbGxhbWEzLTEubGxhbWFtZXRhLm5ldFwvKiIsIkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyOTM3MTg4NX19fV19&Signature=ampJvj885mPgn02YZucFZRpiy8cR9WfRcQKGYX-oGii5scGvu2Gc6LwxeR01Bt3tsbwEseOnOhiGkYbS9NIMxVDQ-7SUsn8tkPMZIEhVmcLZPfaJwB%7EwY8J5T6FQg2PQmp5k2fw6S0RIqcRRc-XDaBC2g-eW73Pr3TGwgyF6V31B-j%7EILLC-272D02U80KovBMtOtXU4CCsRqpD-Pd11w38qV3qCVUDvOzmahkZPVL7akvwGY57Ywe15Ob4oi%7EN1NsH0ExtOwpQ25Q4XKDDw8S85PswP1J6GDKWU8wQAQJEn97KzFnL0LTV0z5g2aGyA7LcMAOMU7ks5G93MVPA4BA__&Key-Pair-Id=K15QRJLYKIFSLZ&Download-Request-ID=1075414637425324


In [1]:
!pip install transformers accelerate huggingface_hub

In [2]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 23.2 MB/s eta 0:00:00:00:010:01m


In [ ]:
import json
import csv
import torch
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
from torch import cuda, bfloat16
import pandas as pd
import google.generativeai as genai
import os
import time
#from groq import Groq
#import openai
from google.api_core.exceptions import InternalServerError
import re
from huggingface_hub import login
from transformers import (AutoModelForCausalLM, AutoModel,
                          AutoTokenizer, 
                          BitsAndBytesConfig, 
                          TrainingArguments, 
                          pipeline, 
                          logging)
login("")


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
#from peft import LoraConfig, PeftConfig
# from trl import SFTTrainer
# from trl import setup_chat_format
from transformers import (AutoModelForCausalLM, AutoModel,
                          AutoTokenizer, 
                          BitsAndBytesConfig, 
                          TrainingArguments, 
                          pipeline, 
                          logging)
from sklearn.metrics import (accuracy_score, 
                             classification_report, 
                             confusion_matrix)
from sklearn.model_selection import train_test_split

from scipy import stats
from sklearn.metrics import roc_curve, auc, roc_auc_score, precision_recall_curve
from sklearn.isotonic import IsotonicRegression       #For calibrating the probs
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn import metrics   #Additional scklearn functions

from sklearn.metrics import confusion_matrix
import plotly.express as px
from sklearn.metrics import RocCurveDisplay



# There are important!
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

In [5]:
import json
import numpy as np
from scipy import spatial

In [6]:
import torch
import json

ArticleEmbedding = []

def compute_article_embedding(embed, tokenizer, batch_size=10, max_length=512):
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    with open("/kaggle/input/dataset-multi-class-classification/NewsBlog_ProthomAlo_FinalDataset.json", "r", encoding='utf-8') as f:
        data = json.loads(f.read())
        
        embed.eval()  # Set the model to evaluation mode
        
        # Process articles in batches to avoid memory overflow
        for l in range(0, 7245, batch_size):
            print(f"Processing batch {l//batch_size + 1}")
            batch_data = data[l:l + batch_size]  # Process a batch of articles
            
            for article_data in batch_data:
                article = {}
                article_text = article_data['Article Text']
                article['Article Text'] = article_text

                # Tokenize with truncation and padding
                inputs = tokenizer(article_text, return_tensors="pt", truncation=True, max_length=max_length, padding="max_length")

                # Get model output without calculating gradients
                with torch.no_grad():
                    output = embed(**inputs, output_hidden_states=True)

                # Extract the last hidden state and apply mean pooling over the tokens
                last_hidden_state = output.hidden_states[-1]  # Use the last hidden state
                article_embedding = torch.mean(last_hidden_state, dim=1).squeeze()  # Mean pooling over the tokens

                # Store the article embedding and label
                article['Article Embedding'] = article_embedding.cpu().numpy()  # Convert tensor to numpy array
                article['Label'] = article_data.get('Article Topics', None)  # Get label if available

                # Append the processed article to the list
                ArticleEmbedding.append(article)
    
    return ArticleEmbedding

# Call the function with your model and tokenizer
# ArticleEmbedding = compute_article_embedding(model, tokenizer)


In [7]:
            # try:
            #     # Your code here
            # except Exception as e:
            #     print(f"An error occurred: {e}")

In [8]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,  #False
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16" #"float16", torch.float16,
    #load_in_8bit_fp32_cpu_offload=True,
)

In [9]:
# import torch

# # Initialize an empty list to store the embeddings
# ArticleEmbedding = []

# def compute_article_embedding(embed, tokenizer):
#     # Load the dataset from the JSON file
#     with open("/kaggle/input/dataset-multi-class-classification/NewsBlog_ProthomAlo_FinalDataset.json", "r") as f:
#         data = json.loads(f.read())
        
#         # Set the model to evaluation mode (disables dropout, etc.)
#         embed.eval()
        
#         # Loop through each article in the dataset
#         for l, article_data in enumerate(data):
#             article = dict()
#             print(f"Processing article number: {l + 1}")
            
#             # Get the article text
#             article_text = article_data['Article Text']
            
#             # Tokenize the article text
#             inputs = tokenizer(article_text, return_tensors="pt", truncation=True, max_length=256)  # Limit to 512 tokens

            
#             # Perform inference without storing gradients
#             with torch.no_grad():
#                 embedding = embed(**inputs, output_hidden_states=True)
#                 hidden_state = embedding.hidden_states[-1]
#             # Take the mean of the embedding along the sequence dimension (dim=1) to get a single vector for the article
#             article_embedding = hidden_state.mean(dim=1).squeeze()
            
#             # Add the embedding and other details to the article dictionary
#             article['Article Text'] = article_text
#             article['Article Embedding'] = article_embedding
#             article['Label'] = article_data.get('Concept')
            
#             # Append the processed article to the list of embeddings
#             ArticleEmbedding.append(article)
#             torch.cuda.empty_cache()
#     return ArticleEmbedding


In [10]:
def compute_label_embedding(embed, tokenizer):
    print("inside label embedding")
    with open("/kaggle/input/dataset-multi-class-classification/Keyword_BengaliNews.json", "r") as f:
        key = json.loads(f.read())
        
        for i in range(len(key)):
            label_embedding = 0
            label = key[i]['Keyword'][0]
            
            for j in range(len(key[i]['Keyword'])):
                # Tokenize the keyword
                inputs = tokenizer(key[i]['Keyword'][j], return_tensors="pt", truncation=True, max_length=512)
                
                # Perform inference without gradients
                with torch.no_grad():
                    # Forward pass through the model
                    outputs = embed(**inputs, output_hidden_states=True)
                    
                    # Extract the last hidden state tensor
                    hidden_state = outputs.hidden_states[-1]  # This gives you the tensor
                    
                    # Mean pooling across the sequence dimension (dim=1)
                    keyword_embedding = hidden_state.mean(dim=1).squeeze()
                    
                    # Accumulate embeddings
                    label_embedding += keyword_embedding
            
            # Store averaged embeddings
            LabelEmbedding[label] = label_embedding / len(key[i]['Keyword'])
    
    return LabelEmbedding


In [11]:
# LabelEmbedding={}
# # label_path=""

# def compute_label_embedding(embed, tokenizer):
#     print("inside label embedding")
#     with open("/kaggle/input/dataset-multi-class-classification/Keyword_BengaliNews.json", "r", encoding='utf-8') as f:
#         key = json.loads(f.read())
#         for i in range(0, len(key)):
#             label_embedding = 0
#             label = key[i]['Keyword'][0]
#             for j in range(0, len(key[i]['Keyword'])):
#                  #p = []
#                  inputs = tokenizer(key[i]['Keyword'][j], return_tensors="pt")
#                  with torch.no_grad():
#                      outputs = embed(**inputs, output_hidden_states=True)
#                      hidden_states = outputs.hidden_states[-1]  # Get the last hidden layer
#                  keyword_embedding = hidden_states.mean(dim=1)  # Mean pooling over sequence length
#                  #print(keyword_embedding)
#                  # Accumulate the embeddings
#                  label_embedding += keyword_embedding
#             LabelEmbedding[label] = label_embedding / len(key[i]['Keyword'])
#     return LabelEmbedding

In [12]:
# import json
# import torch
# from transformers import AutoModel, AutoTokenizer

# LabelEmbedding = []
# batch_size = 1  # Reduce batch size to minimize memory usage

# def compute_label_embedding2(embed, tokenizer, batch_size=10):
#     print("inside label embedding")
    
#     # Set pad token if not already set
#     if tokenizer.pad_token is None:
#         tokenizer.pad_token = tokenizer.eos_token
    
#     with open("/kaggle/input/dataset-multi-class-classification/Keyword_BengaliNews.json", "r") as f:
#         key = json.loads(f.read())
    
#     with open("/kaggle/input/dataset-multi-class-classification/NewsBlog_ProthomAlo_annotated_explicitly.json", "r") as f:
#         data = json.loads(f.read())
    
#     # Clear CUDA memory before starting
#     torch.cuda.empty_cache()

#     for i in range(0, len(key)):
#         count = 0
#         label_embedding = 0
#         label = key[i]['Keyword'][0]
#         print("Working on key ",i)
#         # Process data in batches to avoid memory overload
#         for batch_start in range(0, 7245, batch_size):
#             batch = data[batch_start:batch_start + batch_size]
            
#             for l in batch:
#                 if label in l['Explicit Article Topics']:
#                     inputs = tokenizer([l['Article Text']], return_tensors="pt", padding=True, truncation=True, max_length=512)  # Truncate to 512 tokens
#                     inputs = {key: value.to('cuda') for key, value in inputs.items()}  # Move inputs to GPU
                    
#                     with torch.no_grad():  # Disable gradient calculations for memory savings
#                         outputs = embed(**inputs, output_hidden_states=True)
#                         hidden_states = outputs.hidden_states[-1]
                    
#                     keyword_embedding = hidden_states.mean(dim=1)
#                     label_embedding += keyword_embedding
#                     count += 1
            
#             # Clear CUDA cache after each batch
#             torch.cuda.empty_cache()
        
#         if count > 0:
#             LabelEmbedding[label] = label_embedding / count
#         else:
#             # If no matching articles, calculate from the keyword directly
#             for j in range(0, len(key[i]['Keyword'])):
#                 inputs = tokenizer([key[i]['Keyword'][j]], return_tensors="pt", padding=True, truncation=True, max_length=512)
#                 inputs = {key: value.to('cuda') for key, value in inputs.items()}  # Move inputs to GPU
                
#                 with torch.no_grad():
#                     outputs = embed(**inputs, output_hidden_states=True)
#                     hidden_states = outputs.hidden_states[-1]
                
#                 keyword_embedding = hidden_states.mean(dim=1)
#                 label_embedding += keyword_embedding

#             LabelEmbedding[label] = label_embedding / len(key[i]['Keyword'])

#     return LabelEmbedding

# # Load model and tokenizer
# # model_name = "path_to_your_model"
# # model = AutoModel.from_pretrained(model_name).to('cuda')  # Load model to GPU
# #tokenizer = AutoTokenizer.from_pretrained(model_name)

# # Enable gradient checkpointing to save memory


# # Call the function
# #LabelEmbedding = compute_label_embedding2(model, tokenizer)

# # Clear CUDA memory after processing



In [13]:
dataset_path="/kaggle/input/dataset-multi-class-classification/NewsBlog_ProthomAlo_FinalDataset.json"
result_path="/kaggle/working/output(1000-2000)"

#torch.cuda.empty_cache()
#device_map={"": 0}
#Load the model here
base_model = "BanglaLLM/bangla-llama-13b-instruct-v0.1" 

base_model_name = base_model
model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
    torch_dtype="float16",
    quantization_config=bnb_config, 
)
model.gradient_checkpointing_enable()
torch.cuda.empty_cache()

model.config.use_cache = False
model.config.pretraining_tp = 1
# model = AutoModel.from_pretrained(base_model_name, device_map="auto", offload_folder="offload")
# model.gradient_checkpointing_enable()
# Load the tokenizer here if needed
tokenizer = AutoTokenizer.from_pretrained(base_model_name, use_fast=True, trust_remote_code=True)

#LaBSE implementation
# model = SentenceTransformer('sentence-transformers/LaBSE')

#LASER implementation
# laser = Laser()

# Bangla transformer implementation
# transformer=Bangla_sentence_transformer_small()


#Bloom implementation
# tokenizer = AutoTokenizer.from_pretrained("bigscience/bloomz-1b1")
# model = BloomModel.from_pretrained("bigscience/bloomz-1b1")


#Flan-UL2 implementation
# model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-ul2")
# tokenizer = AutoTokenizer.from_pretrained("google/flan-ul2")

#GPTNeoX implementation
# tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")
# model = GPTNeoXModel.from_pretrained("EleutherAI/gpt-neox-20b")
LabelEmbedding={}
ArticleEmbedding=[]

LabelEmbedding= compute_label_embedding2(model, tokenizer)
ArticleEmbedding= compute_article_embedding(model, tokenizer)



config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/90.4k [00:00<?, ?B/s]

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/9.93G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.99G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/6.99G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/765 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/934k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.77M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

NameError: name 'compute_label_embedding2' is not defined

In [ ]:
import random

In [ ]:


for threshold in np.arange(0.95,0.99,0.05):
        final_article_list = []
        print("Threshold running for::",threshold)
        filename = str(threshold) + '_NewsBlog_ProthomAlo_Dataset.json'
        for i in range(0, len(ArticleEmbedding)):
            inferred_label=[]
            final_article = {}
            for label in LabelEmbedding:
                x = ArticleEmbedding[i]['Article Embedding']
                #.hidden_states[-1]
                label_embedding_np = LabelEmbedding[label].cpu().numpy().flatten()
                squeezed_embedding_np = x.flatten()
                squeezed_embedding_np = [np.mean(squeezed_embedding_np)]
                label_embedding_np = [np.mean(label_embedding_np)]
                # print(label_embedding_np)
                # print(squeezed_embedding_np)
                cosine_dist = 1 - spatial.distance.cosine(label_embedding_np, squeezed_embedding_np)
                #print(cosine_dist)
                if (cosine_dist > threshold):
                    inferred_label.append(label)
            # #random function to pick 5 labels.
            #then send as the output label
            # sample_size = min(len(inferred_label), 8)
            # inferred_label = random.sample(inferred_label, sample_size)
            final_article['Article Text'] = ArticleEmbedding[i]['Article Text']
            final_article['Inferred Article Label'] = inferred_label # inferred labels
            final_article['Label'] = ArticleEmbedding[i]['Label']  # ground truth labels added for evaluation
            final_article_list.append(final_article)

        with open(result_path+filename, 'w', encoding='utf-8', newline='\n') as f:
            #print("writing json::",filename)
            json.dump(final_article_list, f, ensure_ascii=False, indent=4, separators=(',', ':'))
            

In [ ]:
# import json

# # Function to decode Unicode-escaped Bangla text in JSON file
# def decode_bangla_json(input_file, output_file):
#     # Open and read the JSON file
#     with open(input_file, 'r', encoding='utf-8') as f:
#         data = json.load(f)

#     # Save the JSON data with the correct encoding
#     with open(output_file, 'w', encoding='utf-8') as f:
#         json.dump(data, f, ensure_ascii=False, indent=4)

# # Input and output file paths
# input_file = '/kaggle/input/unicoded-bangla/output0.9500000000000001_NewsBlog_ProthomAlo_Dataset.json'  # Replace with your input JSON file path
# output_file = '/kaggle/working/decoded_bangla_output.json'  # Replace with your desired output file path

# # Call the function to decode Unicode-escaped text
# decode_bangla_json(input_file, output_file)

# print(f"Decoded JSON file saved to: {output_file}")
